In [3]:
import mxnet as mx
import os
import random
from mxnet.image import ImageIter

In [2]:
data_dir = "./data/Caltech256/"

In [4]:
!curl -O https://raw.githubusercontent.com/dmlc/mxnet/master/tools/im2rec.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13176  100 13176    0     0  53823      0 --:--:-- --:--:-- --:--:-- 62150


In [5]:
%run -i 'im2rec.py --list=true'

ERROR:root:File `'im2rec.py --list=true.py'` not found.


In [19]:
def list_image(root, recursive, exts):
    image_list = []
    i = 0
    if recursive:
        cat = {}
        for path, subdirs, files in os.walk(root):
            for fname in files:
                fpath = os.path.join(path, fname)
                suffix = os.path.splitext(fname)[1].lower()
                if os.path.isfile(fpath) and (suffix in exts):
                    if path not in cat:
                        label = path.split("/")[-1]
                        cat[path] = int(label) #use the last digit as the class ID#print cat[path]
                    image_list.append((i, os.path.relpath(fpath, root), cat[path]))
                    i += 1
    else:
        for fname in os.listdir(root):
            fpath = os.path.join(root, fname)
            suffix = os.path.splitext(fname)[1].lower()
            if os.path.isfile(fpath) and (suffix in exts):
                image_list.append((i, os.path.relpath(fpath, root), 0))
                i += 1
    return image_list
def make_list(image_list, shuffle=True, chunks=1, prefix="pre", train_ratio=0.65, test_ratio=0.2):
#     image_list = list_image(args.root, args.recursive, args.exts)
#     image_list = list(image_list)
    if shuffle is True:
        random.seed(100)
        random.shuffle(image_list)
    N = len(image_list)
    chunk_size = (N + chunks - 1) // chunks
    for i in range(chunks):
        chunk = image_list[i * chunk_size:(i + 1) * chunk_size]
        if chunks > 1:
            str_chunk = '_%d' % i
        else:
            str_chunk = ''
        sep = int(chunk_size * train_ratio)
        sep_test = int(chunk_size * test_ratio)
        if train_ratio == 1.0:
            write_list(prefix + str_chunk + '.lst', chunk)
        else:
            if test_ratio:
                write_list(prefix + str_chunk + '_test.lst', chunk[:sep_test])
            if train_ratio + test_ratio < 1.0:
                write_list(prefix + str_chunk + '_val.lst', chunk[sep_test + sep:])
            write_list(prefix + str_chunk + '_train.lst', chunk[sep_test:sep_test + sep])
def write_list(path_out, image_list):
    with open(path_out, 'w') as fout:
        for i, item in enumerate(image_list):
            line = '%d\t' % item[0]
            for j in item[2:]:
                line += '%f\t' % j
            line += '%s\n' % item[1]
            fout.write(line)

In [20]:
img_list = list_image("data/Caltech256/", True, ['.jpg', '.jpeg'])

In [21]:
make_list(img_list)

In [31]:
img_iter = ImageIter(batch_size=4, data_shape=(3, 224, 224), label_width=1, path_imglist="data/pre_test.lst", path_root="data/Caltech256/")

loading image list...


In [33]:
batch = next(img_iter)

In [36]:
data = batch.data

In [39]:
data[0].asnumpy()

array([[[[ 255.,  255.,  255., ...,  255.,  255.,  255.],
         [ 255.,  255.,  255., ...,  255.,  255.,  255.],
         [ 255.,  255.,  255., ...,  255.,  255.,  255.],
         ..., 
         [  51.,   24.,   44., ...,  255.,  255.,  255.],
         [  61.,   39.,   25., ...,  255.,  255.,  255.],
         [  57.,   63.,   41., ...,  255.,  255.,  255.]],

        [[ 255.,  255.,  255., ...,  255.,  255.,  255.],
         [ 255.,  255.,  255., ...,  255.,  255.,  255.],
         [ 255.,  255.,  255., ...,  255.,  255.,  255.],
         ..., 
         [  51.,   24.,   44., ...,  255.,  255.,  255.],
         [  61.,   39.,   25., ...,  255.,  255.,  255.],
         [  57.,   63.,   41., ...,  255.,  255.,  255.]],

        [[ 255.,  255.,  255., ...,  255.,  255.,  255.],
         [ 255.,  255.,  255., ...,  255.,  255.,  255.],
         [ 255.,  255.,  255., ...,  255.,  255.,  255.],
         ..., 
         [  51.,   24.,   44., ...,  255.,  255.,  255.],
         [  61.,   39.,